In [1]:
import sys
import sqlite3

import pandas as pd
import praw

In [2]:
reddit_api_access = {}

with open('../reddit_api_data', 'r') as file:
    for line in file:
        key, val = line.split(',')
        reddit_api_access[key] = val.strip()

In [3]:
reddit = praw.Reddit(
    client_id = reddit_api_access['client_id'],
    client_secret = reddit_api_access['client_secret'],
    password = reddit_api_access['password'],
    user_agent = reddit_api_access['user_agent'],
    username = reddit_api_access['username']
)

In [4]:
subreddit = reddit.subreddit('love')
top_posts = subreddit.hot(limit = 200)
format(sys.getsizeof(top_posts) / (1024 ** 2), '.8f')

'0.00005341'

In [5]:
table_name = 'top_200_love_posts_reddit'
conn = sqlite3.connect('reddit_love.db')

cursor = conn.cursor()

cursor.execute(
    f'''
        CREATE TABLE IF NOT EXISTS {table_name}
        (
            id INTEGER PRIMARY KEY, 
            title TEXT, 
            score INTEGER, 
            reddit_post_id INTEGER, 
            url TEXT, 
            num_comments INTEGER,
            body TEXT
        );
    '''
)

for post in top_posts:
    cursor.execute(
        f"""
        INSERT INTO {table_name} (
            title, score, reddit_post_id, url, num_comments, body
        ) VALUES (
            '{post.title}', '{post.score}', '{post.id}', '{post.id}', '{post.num_comments}', '{post.selftext}'
        );
        """
    )

conn.commit()
cursor.execute(f'SELECT * FROM {table_name};')
cursor.fetchall()

OperationalError: near "ve": syntax error